<h1>Overview<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [64]:
import os
import re
import sys
import numpy as np
import pandas as pd

from keras import backend as K
from keras.layers import Dense, Conv1D, LSTM, Dropout, Embedding, Layer
from keras.models import Sequential as SequentialModel
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer as KerasTokenizer

from plotly.offline import iplot, init_notebook_mode
from plotly.graph_objs import Scatter, Layout, Figure

from sklearn.linear_model import LinearRegression

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '../ct')

import load
from preprocess import preprocess
from preprocess import Tokenizer
from preprocess.preprocess import separator_samples

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
def T(h, m=80, g=10, d=100, kN=True ):
    T = m*g/2/h * np.sqrt(h**2 + d**2/4)
    if kN:
        T /= 1000
    return T

In [30]:
T(10)

203.9607805437114

In [31]:
T(11)

186.16619252611207

In [23]:
80*10/2/11 * np.sqrt(11*11 + 100*100/4)

1861.6619252611208

In [13]:
np.sqrt(11*11 + 100*100/4)

51.19570294468082

In [14]:
np.sqrt(11**2 + 100**2/4)

51.19570294468082

In [33]:
T(11)/T(10)

0.9127548542903046

In [36]:
T(11, d=0.1)/T(10, d=0.1)

0.9999978306303938

In [38]:
T(h=10, g=1)

203.9607805437114

In [137]:
def simplified_slackline(length, height_min=None, height_max=None, m=80, g=10, show_linear_model=False):
    if height_min is None:
        height_min = length / 10 / 2
    if height_max is None:
        height_max = 2 * length / 10
    
    x = np.linspace(height_min, height_max, num=100)

    data = []
    
    y = [T(h=x_i, d=length, g=g, m=m) for x_i in x]
    trace = Scatter(x=x,
                    y=y,
                    line=dict(shape='spline',
                              color='#9467bd'),
                    name='Estimated force')
    data.append(trace)

    
    if show_linear_model:
        model = LinearRegression()
        model.fit(x.reshape(-1, 1), np.array(y).reshape(-1, 1))
        y_linear = model.predict(x.reshape(-1, 1)).flatten()
        
        try:
            k_estimate =  (y_linear[-1] - y_linear[0]) / (x[-1] - x[0])
        except:
            k_estimate = None
            
        trace_linear = Scatter(x=x,
                               y=y_linear,
                               opacity=0.7,
                               line=dict(shape='spline',
                                         color='#7f7f7f'),
                               name=f'Linear Regression (k≈{k_estimate:.3f}kN)')
        data.append(trace_linear)
    
    layout = Layout(title=f'Forces in a simplified Slackline. [length={length}m, weight_slackliner={m}kg]',
                    xaxis=dict(title='"height" [m]'),
                    yaxis=dict(title='force [kN]'))
    
    figure = Figure(data=data, layout=layout)
    iplot(figure)

In [138]:
simplified_slackline(length=100, height_min=8, height_max=12, show_linear_model=True)

In [140]:
simplified_slackline(length=100, height_min=3, height_max=5, m=80, show_linear_model=True)